In [1]:
"""For the concepts with a Drugbank ID, retrieve inchi and inchikey from UniChem webservices and insert into sqlite mapping db. 
For the 'errors' in retrieving inchi, check if they are biological and flag them as such in the mapping db"""

"For the concepts with a Drugbank ID, retrieve inchi and inchikey from UniChem webservices and insert into sqlite mapping db. \nFor the 'errors' in retrieving inchi, check if they are biological and flag them as such in the mapping db"

In [2]:
import requests
import json
import sqlite3 as sqlite
import time
import xml.etree.ElementTree as ET

In [3]:
# Set parameters
basedir = '/Users/ines/FAERS_y2'
# Location of the sqlite database used for the mapping process
mapping_process_db = basedir + '/data/interim/201903_drug_mapping_process.db'

In [4]:
# Connect to sqlite db

conn = sqlite.connect(mapping_process_db)
cur = conn.cursor()

In [5]:
# Retrieve Drugbank IDS in mapping db
drugbank_ids = [i[5] for i in cur.execute('select * from drug_concepts where drugbank_id is not NULL').fetchall()]
print(len(set(drugbank_ids)))
drugbank_ids[:5]

2391


['DB05276', 'DB11050', 'DB10317', 'DB10797', 'DB10800']

In [6]:
base_url = 'https://www.ebi.ac.uk/unichem/rest/'

In [7]:
response = requests.get(base_url + 'release/')
response.content.decode()

'[{"xref_count_obsolete":"62369039","source_count":"35","structure_count":"157212958","date":"27-JAN-2019","xref_count_current":"199357631","release":"211","xref_count":"261726670"}]'

In [8]:
# Unichem release date 27-JAN-2019
# Get the following details from the Unichem interface:
# containing Drugbank version 5.1.1, released 03 July 2018
# containing ChEMBL version 24

In [22]:
%%time
# Unichem Get Structure function from web services (source 2 is drugbank)
# This takes some minutes, trying not to overwhelm Unichem, don't see any limits on calls on the website but put some microseconds of sleep just in case

compound_dict = dict()
errors = []

for drugbank_id in set(drugbank_ids):
    
    response = requests.get(base_url + '/structure/{}/2'.format(drugbank_id))
    result = json.loads(response.content.decode())
    
    try:
        standardinchi = result[0]['standardinchi']
        inchikey = result[0]['standardinchikey']
        compound_dict[drugbank_id] = {'standardinchi': standardinchi, 'inchikey': inchikey}
    except (KeyError, IndexError):
        errors.append((drugbank_id, result))
    
    time.sleep(0.2)

CPU times: user 44.8 s, sys: 3.13 s, total: 48 s
Wall time: 11min 50s


In [23]:
len(compound_dict.keys()), len(errors)

(1951, 440)

In [24]:
errors

[('DB00088',
  {'error': " 'DB00088' is not recognized as a src_compound_id from src_id:'2' in UniChem."}),
 ('DB00039',
  {'error': " 'DB00039' is not recognized as a src_compound_id from src_id:'2' in UniChem."}),
 ('DB10504',
  {'error': " 'DB10504' is not recognized as a src_compound_id from src_id:'2' in UniChem."}),
 ('DB08813',
  {'error': " 'DB08813' is not recognized as a src_compound_id from src_id:'2' in UniChem."}),
 ('DB09100',
  {'error': " 'DB09100' is not recognized as a src_compound_id from src_id:'2' in UniChem."}),
 ('DB00072',
  {'error': " 'DB00072' is not recognized as a src_compound_id from src_id:'2' in UniChem."}),
 ('DB11626',
  {'error': " 'DB11626' is not recognized as a src_compound_id from src_id:'2' in UniChem."}),
 ('DB14222',
  {'error': " 'DB14222' is not recognized as a src_compound_id from src_id:'2' in UniChem."}),
 ('DB04895',
  {'error': " 'DB04895' is not recognized as a src_compound_id from src_id:'2' in UniChem."}),
 ('DB11193',
  {'error': " '

In [25]:
# Insert newly obtained inchis and inchikeys into the mapping sqlitedb
for key in compound_dict.keys():
    inchi = compound_dict[key]['standardinchi'] 
    inchikey = compound_dict[key]['inchikey']
    cur.execute("""update drug_concepts set inchi = ?, inchikey = ? where drugbank_id = '{}'""".format(key), (inchi, inchikey))
    
conn.commit()

In [26]:
# Some concepts have errors (no inchi available) because they are biotech / biologicals:
# Check this with Drugbank and if so, add flag in mapping db

In [27]:
# Load drugbank XML files
tree = ET.parse(basedir + '/data/raw/Drugbank_full_database_v5.1.1_2018-07-03.xml')
root = tree.getroot()

In [28]:
drugbank_types = {}
for drug_idx in range(0,len(root)): 
    for i in range(len(root[drug_idx])): # for every element in this item
        if root[drug_idx][i].tag.split("{http://www.drugbank.ca}")[-1] == "drugbank-id":
            drugbank_id = root[drug_idx][i].text
            type = root[drug_idx].attrib["type"]
            drugbank_types[drugbank_id] = type

In [29]:
len(drugbank_types)

16136

In [30]:
# This annotates the type only for the errors

for db_id in [i[0] for i in errors]:
    try:
        type = drugbank_types[db_id]
        cur.execute("""update drug_concepts set drugbank_type = ? where drugbank_id = '{}'""".format(db_id), (type,))
    except KeyError:
        comment = 'not annotated stub'
        cur.execute("""update drug_concepts set drugbank_comment = ? where drugbank_id = '{}'""".format(db_id), (comment,) )

In [31]:
# This annotates the type for the rest

for db_id in compound_dict.keys():
    try:
        type = drugbank_types[db_id]
        cur.execute("""update drug_concepts set drugbank_type = ? where drugbank_id = '{}'""".format(db_id), (type,))
    except KeyError:
        comment = 'not annotated stub'
        cur.execute("""update drug_concepts set drugbank_comment = ? where drugbank_id = '{}'""".format(db_id), (comment,) )

In [32]:
# The number of compounds with a drugbank ID but no inchikey
cur.execute("""select count(*) from drug_concepts where (drugbank_id is not null and inchikey is null)""").fetchall()

[(478,)]

In [33]:
# This corresponds to the Drugbank IDs that are biologicals or not annotated
cur.execute("""select drugbank_type, drugbank_comment, count(*) from drug_concepts where drugbank_type == 'biotech' or drugbank_comment is 'not annotated stub' group by drugbank_type, drugbank_comment""").fetchall()

[(None, 'not annotated stub', 61), ('biotech', None, 267)]

In [35]:
267+61

328

In [36]:
# How many compounds with inchikey
cur.execute('select count(*) from drug_concepts where drugbank_id is not null and inchikey is not null').fetchall()

[(1994,)]

In [37]:
# There are no biologicals with inchikey, as it should be.
cur.execute("select count(*) from drug_concepts where drugbank_type = 'biotech' and inchikey is not null").fetchall()

[(0,)]

In [38]:
# Breakdown of annotations related to Drugbank
cur.execute("select drugbank_comment, drugbank_type, count(drugbank_id) from drug_concepts group by drugbank_comment, drugbank_type").fetchall()

[(None, None, 0),
 (None, 'biotech', 267),
 (None, 'small molecule', 2144),
 ('not annotated stub', None, 61)]

In [39]:
# How many compounds left unannotated?
cur.execute('select count(*) from drug_concepts where drugbank_id is null').fetchall()

[(1054,)]

In [40]:
conn.commit()
conn.close()

In [ ]:
# Not all errors to Drugbank mapping were biologicals, not clear what the reason is